<a href="https://colab.research.google.com/github/esholmess/Skin-Disease-Detection-with-Computer-Vision/blob/main/Healthcare_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kaggle
pip install tensorflow.keras
pip install tensorflow opencv-python pillow pandas scikit-learn streamlit


In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"esholmess","key":"ef45c6fe16ffd0975e923342006a77f4"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle datasets download -d pacificrm/skindiseasedataset --unzip


Dataset URL: https://www.kaggle.com/datasets/pacificrm/skindiseasedataset
License(s): CC0-1.0
 98% 1.33G/1.36G [00:05<00:00, 237MB/s]
100% 1.36G/1.36G [00:05<00:00, 268MB/s]


In [ ]:
import os
import shutil
import random
from pathlib import Path


SOURCE_DIR = "IMG_CLASSES"
TARGET_DIR = "split_dataset"
SPLITS = {'train': 0.7, 'val': 0.2, 'test': 0.1}


for class_name in os.listdir(SOURCE_DIR):
    class_path = os.path.join(SOURCE_DIR, class_name)
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    random.shuffle(images)

    total = len(images)
    train_count = int(SPLITS['train'] * total)
    val_count = int(SPLITS['val'] * total)

    split_data = {
        'train': images[:train_count],
        'val': images[train_count:train_count + val_count],
        'test': images[train_count + val_count:]
    }

    for split in ['train', 'val', 'test']:
        split_dir = os.path.join(TARGET_DIR, split, class_name)
        os.makedirs(split_dir, exist_ok=True)

        for img in split_data[split]:
            src = os.path.join(class_path, img)
            dst = os.path.join(split_dir, img)
            shutil.copy2(src, dst)

print("dataset has succesfully splited")


In [4]:
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
import numpy as np

root = "SkinDisease/SkinDisease"
TARGET_SIZE = (224, 224)

for split in ["train", "test"]:
    split_dir = os.path.join(root, split)

    for class_name in os.listdir(split_dir):
        class_dir = os.path.join(split_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            try:
                with Image.open(img_path) as img:
                    img = img.convert("RGB")
                    img = img.resize(TARGET_SIZE)
                    img.save(img_path)
            except Exception as e:
                print(f"Failed to process {img_path}: {e}")

print("All images resized in-place to 224x224.")

# === 2. LOAD DATASETS ===
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Load before mapping to get class names
raw_train_ds = tf.keras.utils.image_dataset_from_directory(
    "SkinDisease/SkinDisease/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

raw_test_ds = tf.keras.utils.image_dataset_from_directory(
    "SkinDisease/SkinDisease/test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)


class_names = raw_train_ds.class_names
num_classes = len(class_names)
print("Class names:", class_names)


normalization_layer = layers.Rescaling(1./255)
train_ds = raw_train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = raw_test_ds.map(lambda x, y: (normalization_layer(x), y))


base = EfficientNetB2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
out = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base.input, outputs=out)

# Freeze base model
for layer in base.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.fit(train_ds, validation_data=test_ds, epochs=10)

loss, acc = model.evaluate(test_ds)
print(f"\n🧪 Test Accuracy: {acc:.4f}")
print(f"📉 Test Loss: {loss:.4f}")




Exception ignored in: <function _xla_gc_callback at 0x7af3f0df5c60>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


All images resized in-place to 224x224.
Found 13898 files belonging to 22 classes.
Found 1546 files belonging to 22 classes.
Class names: ['Acne', 'Actinic_Keratosis', 'Benign_tumors', 'Bullous', 'Candidiasis', 'DrugEruption', 'Eczema', 'Infestations_Bites', 'Lichen', 'Lupus', 'Moles', 'Psoriasis', 'Rosacea', 'Seborrh_Keratoses', 'SkinCancer', 'Sun_Sunlight_Damage', 'Tinea', 'Unknown_Normal', 'Vascular_Tumors', 'Vasculitis', 'Vitiligo', 'Warts']
31790344/31790344 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 92s 129ms/step - accuracy: 0.1124 - loss: 3.0074 - val_accuracy: 0.1223 - val_loss: 2.9731
Epoch 2/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 26s 59ms/step - accuracy: 0.1196 - loss: 2.9813 - val_accuracy: 0.1223 - val_loss: 2.9716
Epoch 3/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 26s 60ms/step - accuracy: 0.1173 - loss: 2.9751 - val_accuracy: 0.1223 - val_loss: 2.9604
Epoch 4/10
435/435 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - accuracy: 0.1214 - loss: 2.9730 - val_accuracy: 0.12

In [5]:
model.save('skin_disease_model.h5')
print("💾 Model saved as 'skin_disease_model.h5'")

💾 Model saved as 'skin_disease_model.h5'
